for input video - left leg 90 right leg straight

In [16]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
frame_count = 0
total_frames = -1  
# reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
reference_image = cv2.imread(r'dataset/amit_v1_reference.jpg')
threshold = 0.08
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

input_video_path = r'dataset/vtest1.mp4'
# input_video_path = r'dataset/test.mp4'
# input_video = cv2.VideoCapture(0)
input_video = cv2.VideoCapture(input_video_path)
keypoint_labels = {
    "Perfect": "Correct",
    "HandsNotAtRightPosition": "Partially Correct",
    "HandsNotAt90": "Partially Correct",
    "LegNot90": "Partially Correct",
    "LegsWrong": "Incorrect",
    "Idle": "Incorrect",
    "LegNotStraight": "Incorrect",
    "ShoulderWrong": "Incorrect"
}
total_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

correct_frames = 0
partially_correct_frames = 0
incorrect_frames = 0
while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_not_90 = False
    hands_not_at_right_position = False
    hands_not_at_90 = False
    shoulder_wrong = False
    leg_not_straight = False

    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]


        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])
        distance = calculate_distance(adjusted_reference_point, frame_point)
        
        if distance < threshold:
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
        else:
            
            if frame_keypoint_number in [25, 27, 29, 31]:
                leg_not_90 = True
            if frame_keypoint_number in [26, 28, 30, 32]:
                leg_not_straight = True
            if frame_keypoint_number == 13 or frame_keypoint_number == 14:
                hands_not_at_90 = True
            if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                hands_not_at_right_position = True
            if frame_keypoint_number in [11] or frame_keypoint_number in [12]:
                shoulder_wrong = True
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if shoulder_wrong:
        label = keypoint_labels["ShoulderWrong"]
    elif leg_not_90:
        if leg_not_straight:
            if hands_not_at_90:
                label = keypoint_labels["Idle"]
            else:
                label = keypoint_labels["LegsWrong"]
        else:
            label = keypoint_labels["LegNot90"]
    elif leg_not_straight:
        label = keypoint_labels["LegNotStraight"]
    elif hands_not_at_right_position:
        if hands_not_at_90:
            label = keypoint_labels["HandsNotAt90"]
        else:
            label = keypoint_labels["HandsNotAtRightPosition"]

    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Processed Video", frame_copy)
    if label == keypoint_labels["Perfect"]:
        correct_frames += 1
    elif label in [keypoint_labels["HandsNotAtRightPosition"], keypoint_labels["HandsNotAt90"], keypoint_labels["LegNot90"], keypoint_labels["LegNotStraight"]]:
        partially_correct_frames += 1
    elif label in [keypoint_labels["LegsWrong"], keypoint_labels["Idle"], keypoint_labels["ShoulderWrong"]]:
        incorrect_frames += 1
    
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
cv2.destroyAllWindows()
if total_frames == -1:
    total_frames = frame_count
step = 100 / total_frames

score1 = correct_frames*step
score2 = partially_correct_frames*step/2
score3 = incorrect_frames*step/10
final_score = max(0, score1 + score2 - score3)
print(f"Total Frames: {total_frames}")
print(f"Correct Frames: {correct_frames}")
print(f"Partially Correct Frames: {partially_correct_frames}")
print(f"Incorrect Frames: {incorrect_frames}")
print(f"Final Score: {final_score}")

with hands threshold - left leg 90 right leg straight

In [5]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
frame_count = 0
total_frames = -1  
# reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
reference_image = cv2.imread(r'dataset/amit_v_laptop.jpg')
threshold = 0.05
hands_threshold = 0.1  # New hands threshold
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

input_video_path = r'dataset/vtest1.mp4'
input_video = cv2.VideoCapture(0)
# input_video = cv2.VideoCapture(input_video_path)
keypoint_labels = {
    "Perfect": "Correct",
    "HandsNotAtRightPosition": "Partially Correct",
    "HandsNotAt90": "Partially Correct",
    "LegNot90": "Partially Correct",
    "LegsWrong": "Incorrect",
    "Idle": "Incorrect",
    "LegNotStraight": "Incorrect",
    "ShoulderWrong": "Incorrect"
}
total_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

correct_frames = 0
partially_correct_frames = 0
incorrect_frames = 0
while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_not_90 = False
    hands_not_at_right_position = False
    hands_not_at_90 = False
    shoulder_wrong = False
    leg_not_straight = False

    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]

        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])

        # Check for hands using the hands_threshold
        if frame_keypoint_number in [15, 16, 17, 18, 19, 20, 21, 22]:
            distance = calculate_distance(adjusted_reference_point, frame_point)
            if distance < hands_threshold:
                cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
            else:
                hands_not_at_90 = True
                cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)
        else:
            # Check for other keypoints using the general threshold
            distance = calculate_distance(adjusted_reference_point, frame_point)
            if distance < threshold:
                cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
            else:
                if frame_keypoint_number in [25, 27, 29, 31]:
                    leg_not_90 = True
                if frame_keypoint_number in [26, 28, 30, 32]:
                    leg_not_straight = True
                if frame_keypoint_number == 13 or frame_keypoint_number == 14:
                    hands_not_at_90 = True
                # if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                    
                if frame_keypoint_number in [11] or frame_keypoint_number in [12]:
                    shoulder_wrong = True
                cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if shoulder_wrong:
        label = keypoint_labels["ShoulderWrong"]
    elif leg_not_90:
        if leg_not_straight:
            if hands_not_at_90:
                label = keypoint_labels["Idle"]
            else:
                label = keypoint_labels["LegsWrong"]
        else:
            label = keypoint_labels["LegNot90"]
    elif leg_not_straight:
        label = keypoint_labels["LegNotStraight"]
    elif hands_not_at_right_position:
        if hands_not_at_90:
            label = keypoint_labels["HandsNotAt90"]
        else:
            label = keypoint_labels["HandsNotAtRightPosition"]
    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Processed Video", frame_copy)
    if label == keypoint_labels["Perfect"]:
        correct_frames += 1
    elif label in [keypoint_labels["HandsNotAtRightPosition"], keypoint_labels["HandsNotAt90"], keypoint_labels["LegNot90"], keypoint_labels["LegNotStraight"]]:
        partially_correct_frames += 1
    elif label in [keypoint_labels["LegsWrong"], keypoint_labels["Idle"], keypoint_labels["ShoulderWrong"]]:
        incorrect_frames += 1
    
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
cv2.destroyAllWindows()
if total_frames == -1:
    total_frames = frame_count
step = 100 / total_frames

score1 = correct_frames*step
score2 = partially_correct_frames*step/2
score3 = incorrect_frames*step/10
final_score = max(0, score1 + score2 - score3)
print(f"Total Frames: {total_frames}")
print(f"Correct Frames: {correct_frames}")
print(f"Partially Correct Frames: {partially_correct_frames}")
print(f"Incorrect Frames: {incorrect_frames}")
print(f"Final Score: {final_score}")

Total Frames: 44
Correct Frames: 0
Partially Correct Frames: 0
Incorrect Frames: 44
Final Score: 0


live - left leg 90 right leg straight

In [1]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
frame_count = 0
total_frames = -1  
# reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
reference_image = cv2.imread(r'dataset/amit_v_laptop.jpg')
threshold = 0.05
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

input_video_path = r'dataset/vtest1.mp4'
input_video = cv2.VideoCapture(0)
# input_video = cv2.VideoCapture(input_video_path)
keypoint_labels = {
    "Perfect": "Correct",
    "HandsNotAtRightPosition": "Partially Correct",
    "HandsNotAt90": "Partially Correct",
    "LegNot90": "Partially Correct",
    "LegsWrong": "Incorrect",
    "Idle": "Incorrect",
    "LegNotStraight": "Incorrect",
    "ShoulderWrong": "Incorrect"
}
total_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

correct_frames = 0
partially_correct_frames = 0
incorrect_frames = 0
while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_not_90 = False
    hands_not_at_right_position = False
    hands_not_at_90 = False
    shoulder_wrong = False
    leg_not_straight = False

    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]


        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])
        distance = calculate_distance(adjusted_reference_point, frame_point)
        
        if distance < threshold:
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
        else:
            
            if frame_keypoint_number in [25, 27, 29, 31]:
                leg_not_90 = True
            if frame_keypoint_number in [26, 28, 30, 32]:
                leg_not_straight = True
            if frame_keypoint_number == 13 or frame_keypoint_number == 14:
                hands_not_at_90 = True
            if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                hands_not_at_right_position = True
            if frame_keypoint_number in [11] or frame_keypoint_number in [12]:
                shoulder_wrong = True
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if shoulder_wrong:
        label = keypoint_labels["ShoulderWrong"]
    elif leg_not_90:
        if leg_not_straight:
            if hands_not_at_90:
                label = keypoint_labels["Idle"]
            else:
                label = keypoint_labels["LegsWrong"]
        else:
            label = keypoint_labels["LegNot90"]
    elif leg_not_straight:
        label = keypoint_labels["LegNotStraight"]
    elif hands_not_at_right_position:
        if hands_not_at_90:
            label = keypoint_labels["HandsNotAt90"]
        else:
            label = keypoint_labels["HandsNotAtRightPosition"]

    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Processed Video", frame_copy)
    if label == keypoint_labels["Perfect"]:
        correct_frames += 1
    elif label in [keypoint_labels["HandsNotAtRightPosition"], keypoint_labels["HandsNotAt90"], keypoint_labels["LegNot90"], keypoint_labels["LegNotStraight"]]:
        partially_correct_frames += 1
    elif label in [keypoint_labels["LegsWrong"], keypoint_labels["Idle"], keypoint_labels["ShoulderWrong"]]:
        incorrect_frames += 1
    
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
cv2.destroyAllWindows()
if total_frames == -1:
    total_frames = frame_count
step = 100 / total_frames

score1 = correct_frames*step
score2 = partially_correct_frames*step/2
score3 = incorrect_frames*step/10
final_score = max(0, score1 + score2 - score3)
print(f"Total Frames: {total_frames}")
print(f"Correct Frames: {correct_frames}")
print(f"Partially Correct Frames: {partially_correct_frames}")
print(f"Incorrect Frames: {incorrect_frames}")
print(f"Final Score: {final_score}")

Total Frames: 240
Correct Frames: 75
Partially Correct Frames: 61
Incorrect Frames: 104
Final Score: 39.625


live - right leg 90 left leg straight

In [1]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
frame_count = 0
total_frames = -1  
# reference_image = cv2.imread(r'dataset/dhruv_reference.jpg')
reference_image = cv2.imread(r'dataset/amit_v_laptop.jpg')
threshold = 0.05
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

reference_results = pose.process(cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB))

reference_landmarks = []
if reference_results.pose_landmarks:
    for i, landmark in enumerate(reference_results.pose_landmarks.landmark):
        reference_landmarks.append((landmark.x, landmark.y, i))

input_video_path = r'dataset/vtest1.mp4'
input_video = cv2.VideoCapture(0)
# input_video = cv2.VideoCapture(input_video_path)
keypoint_labels = {
    "Perfect": "Correct",
    "HandsNotAtRightPosition": "Partially Correct",
    "HandsNotAt90": "Partially Correct",
    "LegNot90": "Partially Correct",
    "LegsWrong": "Incorrect",
    "Idle": "Incorrect",
    "LegNotStraight": "Incorrect",
    "ShoulderWrong": "Incorrect"
}
total_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

correct_frames = 0
partially_correct_frames = 0
incorrect_frames = 0
while True:
    ret, frame = input_video.read()

    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_results = pose.process(frame_rgb)

    frame_landmarks = []
    if frame_results.pose_landmarks:
        for i, landmark in enumerate(frame_results.pose_landmarks.landmark):
            frame_landmarks.append((landmark.x, landmark.y, i))

    if len(frame_landmarks) > 0 and len(reference_landmarks) > 0:
        initial_offset = np.array(reference_landmarks[0][:2]) - np.array(frame_landmarks[0][:2])

    frame_copy = frame.copy()
    leg_not_90 = False
    hands_not_at_right_position = False
    hands_not_at_90 = False
    shoulder_wrong = False
    leg_not_straight = False

    for frame_landmark in frame_landmarks:
        frame_point = frame_landmark[:2]
        frame_keypoint_number = frame_landmark[2]


        adjusted_reference_point = (reference_landmarks[frame_keypoint_number][0] - initial_offset[0], reference_landmarks[frame_keypoint_number][1] - initial_offset[1])
        distance = calculate_distance(adjusted_reference_point, frame_point)
        
        if distance < threshold:
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 255, 0), -1)
        else:
            
            if frame_keypoint_number in [26, 28, 30, 32]:
                leg_not_90 = True
            if frame_keypoint_number in [25, 27, 29, 31]:
                leg_not_straight = True
            if frame_keypoint_number == 13 or frame_keypoint_number == 14:
                hands_not_at_90 = True
            if frame_keypoint_number in [16, 18, 20, 22] or frame_keypoint_number in [15, 17, 19, 21]:
                hands_not_at_right_position = True
            if frame_keypoint_number in [11] or frame_keypoint_number in [12]:
                shoulder_wrong = True
            cv2.circle(frame_copy, (int(frame_point[0] * frame.shape[1]), int(frame_point[1] * frame.shape[0])), 5, (0, 0, 255), -1)

    if shoulder_wrong:
        label = keypoint_labels["ShoulderWrong"]
    elif leg_not_90:
        if leg_not_straight:
            if hands_not_at_90:
                label = keypoint_labels["Idle"]
            else:
                label = keypoint_labels["LegsWrong"]
        else:
            label = keypoint_labels["LegNot90"]
    elif leg_not_straight:
        label = keypoint_labels["LegNotStraight"]
    elif hands_not_at_right_position:
        if hands_not_at_90:
            label = keypoint_labels["HandsNotAt90"]
        else:
            label = keypoint_labels["HandsNotAtRightPosition"]

    else:
        label = keypoint_labels["Perfect"]

    label_lines = label.split('\n')
    for i, line in enumerate(label_lines):
        cv2.putText(frame_copy, line, (10, 30 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Processed Video", frame_copy)
    if label == keypoint_labels["Perfect"]:
        correct_frames += 1
    elif label in [keypoint_labels["HandsNotAtRightPosition"], keypoint_labels["HandsNotAt90"], keypoint_labels["LegNot90"], keypoint_labels["LegNotStraight"]]:
        partially_correct_frames += 1
    elif label in [keypoint_labels["LegsWrong"], keypoint_labels["Idle"], keypoint_labels["ShoulderWrong"]]:
        incorrect_frames += 1
    
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

input_video.release()
cv2.destroyAllWindows()
if total_frames == -1:
    total_frames = frame_count
step = 100 / total_frames

score1 = correct_frames*step
score2 = partially_correct_frames*step/2
score3 = incorrect_frames*step/10
final_score = max(0, score1 + score2 - score3)
print(f"Total Frames: {total_frames}")
print(f"Correct Frames: {correct_frames}")
print(f"Partially Correct Frames: {partially_correct_frames}")
print(f"Incorrect Frames: {incorrect_frames}")
print(f"Final Score: {final_score}")

KeyError: 'LegNotAbove'

: 